# 12-2 LCEL(LangChain Expression Language)

## 12-3 LCEL 파이프라인 체인 구성

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 단순한 파이프라인 예시
simple_chain = (
    ChatPromptTemplate.from_template("주어진 숫자 {number}를 2진수로 변환해주세요")
    | llm
    | ChatPromptTemplate.from_template("다음 2진수를 16진수로 변환해주세요: {text}")
    | llm
)

# 파이프라인 실행
result = simple_chain.invoke({"number": "42"})
result

AIMessage(content='2진수 `101010`을 16진수로 변환하면 `2A`입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 299, 'total_tokens': 322, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BSI4gNU4iAwCJ3F4Og31GazyYlZ0g', 'finish_reason': 'stop', 'logprobs': None}, id='run-c8494dff-3c35-4cee-a96c-b68b787d943c-0', usage_metadata={'input_tokens': 299, 'output_tokens': 23, 'total_tokens': 322, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 12-4 RunnableParallel을 활용한 병렬 체인 구성

In [4]:
from langchain_core.runnables import RunnableParallel

# 병렬 체인 구성
analysis_chain = RunnableParallel(
    summary=ChatPromptTemplate.from_template("다음 텍스트를 요약해주세요: {text}") | llm,
    sentiment=ChatPromptTemplate.from_template("다음 텍스트의 감정을 분석해주세요: {text}") | llm,
    keywords=ChatPromptTemplate.from_template("다음 텍스트의 주요 키워드를 추출해주세요: {text}") | llm
)

# 모든 분석이 동시에 실행된다.
result = analysis_chain.invoke({
    "text": "오늘은 날씨가 좋아서 공원에서 산책을 했습니다. 많은 사람들이 즐겁게 운동하고 있었어요."
})
result

{'summary': AIMessage(content='오늘 날씨가 좋아 공원에서 산책을 했고, 많은 사람들이 운동을 즐기고 있었습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 43, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BSI5zeQOS8285tSBVN1cj0u7TbHP1', 'finish_reason': 'stop', 'logprobs': None}, id='run-17b349c3-521b-4490-a022-d69d71469b26-0', usage_metadata={'input_tokens': 43, 'output_tokens': 25, 'total_tokens': 68, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'sentiment': AIMessage(content='이 텍스트는 긍정적인 감정을 표현하고 있습니다. "날씨가 좋아서"와 "즐겁게 운동하고 있었어요"라는 표현에서 기분 좋은 분위기와 사람들의 즐거운 활동이 잘 드러납니다. 전반적으로 행복하고 긍정적인 경험을 담고 있는 내용입니다.', addition

## 12-5 RunnablePassthrough를 활용한 중간 결과 전달    

In [5]:
from langchain.schema.runnable import RunnablePassthrough

# 중간 결과를 활용하는 체인
analysis_chain = (
    {
        "original": RunnablePassthrough(),  # 원본 입력을 보존
        "summary": ChatPromptTemplate.from_template("{text}를 한 문장으로 요약해주세요") | llm
    }
    | ChatPromptTemplate.from_template("""
원본 텍스트: {original}
요약: {summary}

위 내용에 대한 분석 리포트를 작성해주세요.
    """)
    | llm
)

# 체인 실행
result = analysis_chain.invoke({
    "text": "LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다. 다양한 컴포넌트를 제공하여 개발을 용이하게 합니다."
})
result

AIMessage(content='## 분석 리포트\n\n### 1. 원본 텍스트\n- **내용**: LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다. 다양한 컴포넌트를 제공하여 개발을 용이하게 합니다.\n- **주요 포인트**:\n  - LangChain은 LLM(대형 언어 모델) 애플리케이션 개발을 위한 프레임워크로, 개발자에게 다양한 컴포넌트를 제공하여 개발 과정을 간소화합니다.\n\n### 2. 요약\n- **내용**: LangChain은 LLM 애플리케이션 개발을 위한 다양한 컴포넌트를 제공하는 프레임워크입니다.\n- **변경 사항**:\n  - 원본 텍스트의 핵심 정보를 유지하면서 문장을 간결하게 재구성하였습니다.\n  - "개발을 용이하게 합니다"라는 표현이 "다양한 컴포넌트를 제공하는"으로 대체되어, 요약이 더 명확하고 직관적으로 전달됩니다.\n\n### 3. 메타데이터 분석\n- **토큰 사용량**:\n  - **총 토큰 수**: 77\n    - **프롬프트 토큰**: 49\n    - **완료 토큰**: 28\n- **모델 정보**:\n  - **모델 이름**: gpt-4o-mini-2024-07-18\n  - **시스템 지문**: fp_0392822090\n- **완료 이유**: \'stop\' - 모델이 자연스럽게 응답을 마쳤음을 나타냅니다.\n\n### 4. 사용 메타데이터\n- **입력 토큰**: 49\n- **출력 토큰**: 28\n- **입력 토큰 세부사항**: \n  - 오디오 관련 토큰: 0\n  - 캐시 읽기 토큰: 0\n- **출력 토큰 세부사항**: \n  - 오디오 관련 토큰: 0\n  - 추론 관련 토큰: 0\n\n### 5. 결론\n- LangChain에 대한 설명이 명확하고 간결하게 요약되었습니다. \n- 메타데이터는 모델의 성능과 응답의 효율성을 보여주며, 입력과 출력의 토큰 수가 적절하게 관리되고 있음을 나타냅니다. \n- 전반적으로, 원본 텍스트의 핵심 메시지를 효과적으로 전달하는 요약이 

## 12-6 retriever 선언

In [6]:
# 필요한 라이브러리들을 가져온다
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
from langchain_chroma import Chroma

# 환경 변수 파일(.env)을 불러온다.
load_dotenv()

# 업스테이지의 SOLAR 임베딩 모델을 초기화한다.
embedding = UpstageEmbeddings(
    model='solar-embedding-1-large'  
)

# Chroma 벡터 저장소를 초기화한다.
vector_store = Chroma(
                  embedding_function=embedding,
                  collection_name='tax-markdown', 
                  persist_directory="./tax-markdown")

# retriever를 설정하고 상위 3개 결과를 반환하도록 한다.
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

## 12-7 LangChain허브에서 프롬프트 불러오기     

In [7]:
# 필요한 패키지 import
from langchain import hub

# LangChain 허브에서 RAG 프롬프트 템플릿을 가져온다.
rag_prompt = hub.pull("rlm/rag-prompt")

# 여러 문서들을 하나의 문자열로 결합하는 헬퍼 함수다.
# 각 문서는 두 줄의 개행으로 구분되어 LLM이 문맥을 더 잘 파악할 수 있다.
def format_docs(docs):
   return "\n\n".join(doc.page_content for doc in docs)


/Users/jasonkang/personal/langgraph-book/.venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## 12-8 LCEL을 활용한 과세 표준 체인 생성     

In [8]:
# LCEL의 기본 컴포넌트들을 가져온다.
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# RAG 체인을 구성한다.
# 1. retriever로 문서를 가져와서 포맷팅한다.
# 2. 프롬프트에 문서와 질문을 전달한다.
# 3. LLM으로 답변을 생성한다.
# 4. 문자열로 파싱한다.
tax_base_chain = (
   {"context": retriever | format_docs, "question": RunnablePassthrough()}
   | rag_prompt
   | llm
   | StrOutputParser()
)

## 12-9 과세 표준 체인 실행

In [10]:
# tax_base_chain 활용을 위한 질문 작성
tax_base_question = "주택에 대한 종합부동산세 과세표준을 계산하는 방법은 무엇인가요?"

# tax_base_chain 실행
tax_base_response = tax_base_chain.invoke(tax_base_question)
tax_base_response

'주택에 대한 종합부동산세 과세표준은 납세의무자가 보유한 주택의 공시가격을 합산한 금액에서 일정 금액(1세대 1주택자: 12억 원, 그 외: 9억 원 등)을 공제한 후, 공정시장가액비율(60~100%)을 곱하여 계산합니다. 공제 금액과 비율은 납세자의 상황에 따라 다릅니다.'

## 12-10 과세 표준 체인 프롬프트 개선

In [ ]:
# tax_base_chain 활용을 위한 프롬프트 수정
tax_base_question = "주택에 대한 종합부동산세 과세표준을 계산하는 방법을 수식으로 표현해서 수식만 반환해주세요. 부연설명을 하지 말아주세요"

# tax_base_chain 실행
tax_base_response = tax_base_chain.invoke(tax_base_question)
tax_base_response

'주택에 대한 종합부동산세 과세표준 = (주택의 공시가격 합산 - 공제금액) × 공정시장가액비율'

## 12-11 공제액 계산 체인 생성 및 활용

In [15]:
# 필요한 라이브러리들을 가져온다.
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# LangChain Hub에서 RAG 프롬프트 템플릿을 가져온다.
rag_prompt = hub.pull("rlm/rag-prompt")

# 검색된 문서들을 하나의 텍스트로 결합하는 함수를 정의한다.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 종합부동산세 공제액 계산을 위한 RAG 체인을 구성한다.
tax_deductible_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

# 공제액 관련 질문을 정의하고 체인을 실행한다.
deductible_question = "주택에 대한 종합부동산세 과세표준의 공제액을 알려주세요"
tax_deductible_response = tax_deductible_chain.invoke(deductible_question)
tax_deductible_response

/Users/jasonkang/personal/langgraph-book/.venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'주택에 대한 종합부동산세 과세표준의 공제액은 1세대 1주택자의 경우 12억 원, 법인 또는 법인으로 보는 단체의 경우 0원, 그 외의 경우 9억 원입니다.'

## 12-12 중간값을 활용한 LCEL 체인 실행

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# 종부세 공제액 계산을 위한 프롬프트 템플릿을 정의한다.
# Context에는 주택 수별 공제액 정보가 들어가고, Question에는 사용자의 질문이 들어간다.
# 프롬프트는 금액만 반환하도록 명확히 지시한다.
question = "10억짜리 집을 2채 가지고 있을 때 세금을 얼마나 내나요?"

user_deduction_prompt = """아래 [Context]는 주택에 대한 종합부동산세의 공제액에 관한 내용입니다.
사용자의 질문을 통해서 가지고 있는 주택수에 대한 공제액이 얼마인지 금액만 반환해주세요

[Context]
{tax_deductible_response}

[Question]
질문: {question}
답변:
"""

# PromptTemplate을 사용해 프롬프트의 변수 부분을 정의한다.
user_deduction_prompt_template = PromptTemplate(
   template=user_deduction_prompt,
   input_variables=['tax_deductible_response', 'question']
)

# 프롬프트 템플릿, LLM, 출력 파서를 연결하여 체인을 구성한다.
user_deduction_chain = (user_deduction_prompt_template
   | llm
   | StrOutputParser()
)

# 체인을 실행하여 사용자의 질문에 대한 공제액을 계산한다.
user_deduction = user_deduction_chain.invoke({
   'tax_deductible_response': tax_deductible_response,
   'question': question
})

# 계산된 공제액을 반환한다.
user_deduction


'9억원'

In [18]:
!uv add -q duckduckgo-search langchain-community

## 12-13 DuckDuckGo를 활용한 웹 검색 도구 활용   

In [19]:

from datetime import datetime
from langchain_community.tools import DuckDuckGoSearchRun

# 덕덕고 검색 도구를 초기화한다.
search = DuckDuckGoSearchRun()

# 현재 연도의 공정시장가액비율을 검색한다.
# datetime.now()로 현재 연도를 동적으로 가져와서 검색어에 포함한다.
market_value_rate_search = search.invoke(f"{datetime.now().year}년도 공정시장가액비율은?")

# 검색 결과를 반환한다.
market_value_rate_search


"공정시장가액비율은 개인주택에만 적용되고 법인주택 및 토지분에는 적용되지 않으며, 공정시장가액비율은 재산세 적용기준이며 2024년 기준으로 공정시장가액비율은 60%이며 2025년에도 동일 적용이 예상됩니다. 공정가액시장비율이라 함은 재산세 과세표준을 구할 때. 주택의 공시가격을 전부 반영하지 않고. 일정 비율만 반영해서 세금을 과세하겠다는 것입니다. 일반적으로 공정가액시장비율은 60%지만 1주택자는 43~45%를 적용합니다. ※ 1세대 1주택 공정가액시장비율. 아파트 공시가격은 아래 링크로 들어가시면 확인하실 수 있습니다. https://www.realtyprice.kr/notice/main/mainBody.htm. 홍길동 씨의 재산세를 한 번 계산해 보죠. 부동산 공시가격 10억이었습니다. 공정시장가액비율을 45% 적용 받게 됩니다. 이 비율은 9억이 넘어도 받을 수 있어요. 특례세율을 적용할 수 있어요. 오늘은 2025년 공동주택 공시가격과 뜻, 재산세 계산 방법 등에 대해 알아봤습니다. 공시가격, 기준시가, 공시지가의 차이와 조회 방법도 잘 알아두시기 바라며, 공시가격에 따른 세금 및 각 종 복지정책에도 영향이 있다는 점도 기억해두시기 바랍니다. 정부가 2025년에도 1주택자에 대한 재산세 경감 조치를 유지 하기로 했습니다. 행정안전부는 1주택자의 재산세 부담을 줄이기 위해 '공정시장가액비율 특례'를 1년 연장 한다고 밝혔다. 공시가격에 따라 적용되는 비율은 3억 이하 43% 3~6억 44% 6억 초과 45%로 유지된다. 이 특례로 인해, 예를 들어 공시가 4억 원 주택 보유자의 재산세는 기존 30만 원에서 약 17.2만 원으로 경감 된다. 또한, 인구감소지역 내 산업용 토지에 대해서는 2029년까지 5년간 재산세 분리과세 (0.2%)가 적용, 종합부동산세 면제 등 세 부담 완화 조치가 시행된다. 바로 2025년에도 공정시장가액비율 특례가 계속 적용된다는 정부 발표가 있었는데요. 실거주 중심의 1세대 1주택자라면 곡 알아두셔야 할 중요한 내용입니다. 1.

## 12-14 공정시장가액비율 계산을 위한 체인

In [20]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 공정시장가액비율 추출을 위한 프롬프트 템플릿을 정의한다.
# Context에는 검색된 정보가 들어가고, 사용자의 질문을 바탕으로 해당하는 비율만 반환한다.
# 프롬프트는 부가 설명 없이 비율만 반환하도록 명확히 지시한다.
market_value_rate_prompt = PromptTemplate.from_template("""아래 [Context]는 공정시장가액비율에 관한 내용입니다.
당신에게 주어진 공정시장가액비율에 관한 내용을 기반으로, 사용자의 상황에 대한 공정시장가액비율을 알려주세요.
별도의 설명 없이 공정시장가액비율만 알려주세요.

[Context]
{context}

[Question]
질문: {question}
답변:
""")

# 프롬프트 템플릿, LLM, 출력 파서를 연결하여 체인을 구성한다.
market_value_rate_chain = (
   market_value_rate_prompt
   | llm
   | StrOutputParser()
)

# 체인을 실행하여 검색 결과에서 사용자 상황에 맞는 공정시장가액비율을 추출한다.
market_value_rate = market_value_rate_chain.invoke({'context': market_value_rate_search, 'question': question})
market_value_rate


'60%'

## 12-15 체인 실행 결과 종합으로 최종 답변 생성

In [21]:
from langchain_core.prompts import ChatPromptTemplate

# 챗봇 형식의 프롬프트 템플릿을 생성한다.
# system 메시지에는 세금 계산에 필요한 모든 기준 정보를 포함한다.
# human 메시지에는 사용자의 질문이 들어간다.
house_tax_prompt = ChatPromptTemplate.from_messages([
   ('system', f'''과세표준 계산방법: {tax_base_response}
공정시장가액비율: {market_value_rate}
공제액: {tax_deductible_response}

위의 공식과 아래 세율에 관한 정보를 활용해서 세금을 계산해주세요.
세율: {{tax_rate}}
'''),
   ('human', '{question}')
])

house_tax_chain = (
   {
       'tax_rate': retriever | format_docs,
       'question': RunnablePassthrough()
   }
   | house_tax_prompt
   | llm
   | StrOutputParser()
)

# 체인을 실행하여 최종 세금을 계산한다.
house_tax = house_tax_chain.invoke(question)
house_tax


'10억 원짜리 집을 2채 소유하고 있는 경우, 납세의무자는 2주택 이하의 소유자로 분류됩니다. 따라서, 주택에 대한 종합부동산세를 계산하기 위해 다음 단계를 따릅니다.\n\n1. **주택의 공시가격 합산**: \n   - 10억 원 + 10억 원 = 20억 원\n\n2. **과세표준 계산**:\n   - 공제액: 1세대 2주택자의 경우 9억 원이 공제됩니다.\n   - 과세표준 = (주택의 공시가격 합산 - 공제금액) × 공정시장가액비율\n   - 과세표준 = (20억 원 - 9억 원) × 0.6 = 11억 원 × 0.6 = 6.6억 원\n\n3. **세액 계산**:\n   - 과세표준 6.6억 원에 해당하는 세율을 적용합니다.\n   - 6억 원 초과 12억 원 이하의 세율을 적용합니다:\n     - 세액 = 360만 원 + (6억 원을 초과하는 금액의 1천분의 10)\n     - 6.6억 원 - 6억 원 = 0.6억 원 = 6천만 원\n     - 세액 = 360만 원 + (6천만 원 × 0.001) = 360만 원 + 60만 원 = 420만 원\n\n따라서, 10억 원짜리 집을 2채 소유하고 있을 때 납부해야 할 종합부동산세는 **420만 원**입니다.'